In [171]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import os
from scipy import optimize
hbarc = 0.1973

In [172]:
def boostpimunu(vx, vy, vz, pimunu):
    '''
    This function use the Lorentz transformation to boost the pimunu tensor 
from the lab frame to the local rest frame of the fluid element
    The input pimunu should be 4*4 matrix  
    '''
    gamma = 1/np.sqrt(1-(vx**2)-(vy**2)-(vz**2))
    lambdatensor = [[gamma, -gamma*vx, -gamma*vy, -gamma*vz],
                    [-gamma*vx, 1+((gamma**2)*(vx**2)/(1+gamma)), (gamma**2)*vx*vy/(1+gamma), (gamma**2)*vx*vz/(1+gamma)],
                    [-gamma*vy, (gamma**2)*vy*vx/(1+gamma), 1+((gamma**2)*(vy**2)/(1+gamma)), (gamma**2)*vy*vz/(1+gamma)],
                    [-gamma*vz, (gamma**2)*vz*vx/(1+gamma), (gamma**2)*vz*vy/(1+gamma), 1+((gamma**2)*(vz**2)/(1+gamma))]]
    piab = np.einsum('mr,ns,rs -> mn', lambdatensor, lambdatensor, pimunu)
    return piab

In [173]:
def diagpiij(piab):
    piij = piab[1:,1:]
    eigvalue, matrix = linalg.eig(piij)
    return eigvalue, matrix

In [174]:
hbar = 0.1973
def equil_eps(beta):
    return 3. / (np.pi**2. * hbar**3. * beta**4.)

def equil_beta(eps):
    a = 3. / (eps * np.pi**2. * hbar**3.)
    return a**(0.25)

def beta_pi(eps):
    beta = equil_beta(eps)
    I_1_42 = 4. / (5. * np.pi**2. * hbar**3. * beta**5.)
    return beta * I_1_42


In [175]:
# ed = np.fromfile('./hydro_info_1/ed.dat')
# pi11 = np.fromfile('./hydro_info_1/pi11.dat')
# pi12 = np.fromfile('./hydro_info_1/pi12.dat')
# pi22 = np.fromfile('./hydro_info_1/pi22.dat')
# u1 = np.fromfile('./hydro_info_1/u1.dat')
# u2 = np.fromfile('./hydro_info_1/u2.dat')
# size = np.int64(np.sqrt(np.shape(u1)[0]))

# ed = ed.reshape((size,size))
# pi11 = pi11.reshape((size,size))
# pi12 = pi12.reshape((size,size))
# pi22 = pi22.reshape((size,size))
# u1 = u1.reshape((size,size))
# u2 = u2.reshape((size,size))
# u3 = np.zeros((size,size))
# u0 = np.sqrt(1 + u1*u1 + u2*u2)
# vx = u1/u0
# vy = u2/u0
# pi00 = (u1*u1*pi11 + u2*u2*pi22 + 2*u1*u2*pi12)/(u0*u0)
# pi33 = pi00 - pi11 - pi22
# pi01 = (pi11*u1 + pi12*u2)/u0
# pi02 = (pi12*u1 + pi22*u2)/u0
# print(np.shape(ed))
# print(np.max(ed))
# print(np.argmax(ed))

In [176]:
time_str = "_0010"
hbarc = 0.1973
ed = np.loadtxt("hydro_info_1/Edlq"+time_str+".txt")*hbarc
vx = np.loadtxt("hydro_info_1/Vxlq"+time_str+".txt")
vy = np.loadtxt("hydro_info_1/Vylq"+time_str+".txt")
pi11 = np.loadtxt("hydro_info_1/Pi11"+time_str+".txt")*hbarc
pi12 = np.loadtxt("hydro_info_1/Pi12"+time_str+".txt")*hbarc
pi22 = np.loadtxt("hydro_info_1/Pi22"+time_str+".txt")*hbarc
bulk = np.loadtxt("hydro_info_1/PPiq"+time_str+".txt")*hbarc
pi00 = (vx*vx*pi11 + vy*vy*pi22 + 2*vx*vy*pi12)*hbarc
pi33 = (pi00 - pi11 - pi22)*hbarc
pi01 = (pi11*vx + pi12*vy)*hbarc
pi02 = (pi12*vx + pi22*vy)*hbarc
print(np.shape(ed))
coords = (np.where(bulk > 1e-8))
for coord in zip(coords[0], coords[1]):
    print(bulk[coord])

(125, 125)


In [177]:
def calij(i,j):
    vxij = vx[i,j]
    vyij = vy[i,j]
    vzij = 0
    pimunu = np.array([[pi00[i,j],pi01[i,j],pi02[i,j],0],
                       [pi01[i,j],pi11[i,j],pi12[i,j],0],
                       [pi02[i,j],pi12[i,j],pi22[i,j],0],
                       [ 0,        0,        0,       pi33[i,j]]])
    #umunu = [u0[i,j],-u1[i,j],-u2[i,j],0]
    #print(np.einsum('ij,j -> i',pimunu,umunu))
    LRFpimunu = (boostpimunu(vxij, vyij, vzij, pimunu))
    #print(LRFpimunu)
    einvalue, einvector = (diagpiij(LRFpimunu))
    this_e = ed[i,j]
    #this_p = p[i,j]
    this_pi1 = einvalue[0]
    this_pi2 = einvalue[1]
    #this_e = 0.3
    #this_p = p[i,j]
    #this_pi1 = -0.035
    #this_pi2 = -0.03

    #print(this_e,this_pi1,this_pi2)
    beta_guess = equil_beta(this_e)
    beta_pi_guess = beta_pi(this_e)
    guess_fac = -beta_guess / (2. * beta_pi_guess)
    #guess_fac *= 0.95 #see if this improves performance
    g1_guess = guess_fac * this_pi1 
    g2_guess = guess_fac * this_pi2
    

    cmd = "./para_solve/solve.exe {} {} {} {} {} {} ".format(this_e, this_pi1.real, this_pi2.real, beta_guess, g1_guess.real, g2_guess.real)
    data = os.popen(cmd)
    strs = data.read()
    print("e: {}, pi1: {}, pi2: {}".format(this_e, this_pi1.real, this_pi2.real))
    print("guess beta: {}, gamma1: {}, gamma2: {}".format(beta_guess, g1_guess.real, g2_guess.real))
    res = strs.split()
    res_fin = [float(num) for num in res]
    print("beta_f0: {}".format(np.power((3/(np.pi*np.pi*(0.1973**3)*this_e)),1/4)))
    print("real beta: {}, gamma1: {}, gamma2: {}".format(res_fin[0],res_fin[1],res_fin[2]))
    print("relative err: {}%, {}%, {}%".format(res_fin[3]*100,res_fin[4]*100,res_fin[5]*100))
    print("\n")
    #return strs

In [178]:
grid = np.shape(ed)
for i in range(grid[0]):
    for j in range(grid[1]):
        if (ed[i,j] > 0.3):
           calij(i,j) 

e: 0.3000933, pi1: 0.046015055861374145, pi2: 0.021855130675059127
guess beta: 3.388800391242593, gamma1: -0.9742959896101968, gamma2: -0.4627478065715211
beta_f0: 3.388800391242593
real beta: 4.85633778, gamma1: -1.84169997, gamma2: -1.5011075
relative err: -1.2569300000000002e-05%, -1.93237e-05%, -2.6164499999999997e-06%


e: 0.31015560000000003, pi1: 0.04561346750808064, pi2: 0.02203475088746581
guess beta: 3.360974029489228, gamma1: -0.9267869076618904, gamma2: -0.44770809481819746
beta_f0: 3.360974029489228
real beta: 4.71532109, gamma1: -1.68436781, gamma2: -1.35827327
relative err: -6.74539e-06%, -2.00504e-05%, 1.70321e-05%


e: 0.31883680000000003, pi1: 0.04583831277446021, pi2: 0.022488124283120407
guess beta: 3.337858738600549, gamma1: -0.8997656453293886, gamma2: -0.44142204267874263
beta_f0: 3.337858738600549
real beta: 4.64015708, gamma1: -1.60976488, gamma2: -1.29458595
relative err: -1.7462699999999998e-05%, -4.45099e-05%, 2.77385e-05%


e: 0.3257423, pi1: 0.046499601744

In [179]:
def calsurface(e,vxij,vyij,vzij,pimunu):
    #umunu = [u0[i,j],-u1[i,j],-u2[i,j],0]
    #print(np.einsum('ij,j -> i',pimunu,umunu))
    LRFpimunu = (boostpimunu(vxij, vyij, vzij, pimunu))
    #print(LRFpimunu)
    einvalue, einvector = (diagpiij(LRFpimunu))
    this_e = e
    #this_p = p[i,j]
    this_pi1 = einvalue[0]
    this_pi2 = einvalue[1]
    #this_e = 0.3
    #this_p = p[i,j]
    #this_pi1 = -0.035
    #this_pi2 = -0.03

    #print(this_e,this_pi1,this_pi2)
    beta_guess = equil_beta(this_e)
    beta_pi_guess = beta_pi(this_e)
    guess_fac = -beta_guess / (2. * beta_pi_guess)
    #guess_fac *= 0.95 #see if this improves performance
    g1_guess = guess_fac * this_pi1 
    g2_guess = guess_fac * this_pi2
    
 
    cmd = "./para_solve/solve.exe {} {} {} {} {} {} ".format(this_e, this_pi1.real, this_pi2.real, beta_guess, g1_guess.real, g2_guess.real)
    data = os.popen(cmd)
    strs = data.read()
    print("e: {}, pi1: {}, pi2: {}".format(this_e, this_pi1.real, this_pi2.real))
    print("guess beta: {}, gamma1: {}, gamma2: {}".format(beta_guess, g1_guess.real, g2_guess.real))
    print("beta_f0: {}".format(np.power((3/(np.pi*np.pi*(0.1973**3)*this_e)),1/4)))
    res = strs.split()
    res_fin = [float(num) for num in res]
    print("real beta: {}, gamma1: {}, gamma2: {}".format(res_fin[0],res_fin[1],res_fin[2]))
    print("relative err: {}%, {}%, {}%".format(res_fin[3]*100,res_fin[4]*100,res_fin[5]*100))
    print("\n")

In [180]:
# surface_data = np.fromfile('./hydro_info_1/surface.dat').reshape(-1,16)
# surface_number = np.shape(surface_data)[0]
# for i in range(0,surface_number):
#     surf_info = (surface_data.reshape(-1,16)[i])
#     edij = 0.26
#     vxij = surf_info[6]
#     vyij = surf_info[7]
#     vzij = 0
#     pimunu = np.array([[surf_info[8],surf_info[9],surf_info[10],0],
#                     [surf_info[9],surf_info[11],surf_info[12],0],
#                     [surf_info[10],surf_info[12],surf_info[13],0],
#                     [0             , 0           ,           0, surf_info[14]]])
#     bulkij = surf_info[15]
#     calsurface(edij,vxij,vyij,vzij,pimunu)
    

In [181]:
def caltest(e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz):
    pi_zz = -(pi_xx + pi_yy) #traceless

    #symmetric matrix
    pi_ij = np.array( [[pi_xx, pi_xy, pi_xz],[pi_xy, pi_yy, pi_yz],[pi_xz, pi_yz, pi_zz]] )

    w, v = linalg.eig(pi_ij) #w are eig vals, v is matrix of eig vectors
    #print("v.T @ v = ")
    #print(v.T @ v)

    pi_1, pi_2 = w[0], w[1] #first two eig vals
    print(e,pi_1,pi_2)
    beta_guess = equil_beta(e)
    beta_pi_guess = beta_pi(e)
    guess_fac = -beta_guess / (2. * beta_pi_guess)
    #guess_fac *= 0.95 #see if this improves performance
    g1_guess = guess_fac * pi_1 
    g2_guess = guess_fac * pi_2 
    guess = [beta_guess, g1_guess, g2_guess]
    print("guess = " + str(guess))
    cmd = "./para_solve/solve.exe {} {} {} {} {} {} ".format(e, pi_1.real, pi_2.real, beta_guess, g1_guess.real, g2_guess.real)
    print(cmd)
    data = os.popen(cmd)
    strs = data.read()
    print(strs)
    res_s = (strs.split('\n')[-2]).split()
    res = [float(num) for num in res_s]
    res = np.array(res)
    beta = res[0]
    g1 = res[1]
    g2 = res[2]
    g3 = -g1-g2
    gamma_ij_D = np.array( [[g1, 0., 0.],[0., g2, 0.],[0., 0., g3]] )
    #now rotate gamma out of the eigenvector basis
    #gamma_ij = (v.T)@(gamma_ij_D)@(v)
    print(v)
    gamma_ij = (v)@(gamma_ij_D)@(v.T)

    return beta, gamma_ij

In [182]:
T_sw = 0.15
beta_sw = 1./T_sw
e = equil_eps(beta_sw)
print(e)
#Conformal EoS
p_eq = e / 3.

R_pi_inv = 1/5.

#shear tensor
pi_xx = p_eq * R_pi_inv
pi_zz = pi_xx
pi_xy = 0.
pi_xz = 0.
pi_yz = 0.
pi_yy = -(pi_xx + pi_zz)

pi_ij = np.array( [[pi_xx, pi_xy, pi_xz],[pi_xy, pi_yy, pi_yz],[pi_xz, pi_yz, pi_zz]] )



beta, gamma_ij = caltest(e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz)
print(beta)
print(gamma_ij)

0.02003573550196675
0.02003573550196675 (0.0013357157001311166+0j) (-0.0026714314002622332+0j)
guess = [6.666666666666667, (-0.8333333333333336+0j), (1.6666666666666672-0j)]
./para_solve/solve.exe 0.02003573550196675 0.0013357157001311166 -0.0026714314002622332 6.666666666666667 -0.8333333333333336 1.6666666666666672 
 8.27257357  -1.29971242  2.59894174  2.98623e-08 -3.39855e-07  1.87093e-07 

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
8.27257357
[[-1.29971242  0.          0.        ]
 [ 0.          2.59894174  0.        ]
 [ 0.          0.         -1.29922932]]


In [183]:
def checkcondition(strs):        
    strslist = strs.splitlines()
    for strs in strslist:
        this_res = strs.split()
        final_res = [float(num) for num in this_res]
        final_res = np.array(final_res)
        para = final_res[0:3]
        def cc(x):
            lambda_e = para[0]
            gamma_1 = para[1]
            gamma_2 = para[2]
            temp_res = (lambda_e - (gamma_1+gamma_2)*x[0] + 
                (gamma_1*x[1] + gamma_2*(1-x[1]))*(1-x[0]))
            return temp_res
        minimum = optimize.dual_annealing(cc,bounds=((0,1),(0,1)))
        
        if (minimum.fun > 0) and (minimum.success == True):
            print(para)
            

In [184]:
#checkcondition(calij(60,53))